In [ ]:
import os
from dotenv import load_dotenv
import json
import pandas as pd
import requests
import time

In [ ]:
load_dotenv()
beakey = os.environ.get('API_KEY_BEA')

In [ ]:
method = "GetData"
data_set = "Regional"
tablename='SAINC1'
geo='STATE'
lc='3'
year='2022'
params = f'TableName={tablename}&GeoFips={geo}&LineCode={lc}&Year={year}'


In [ ]:
data = requests.get(
        f'https://apps.bea.gov/api/data?&UserID={beakey}' +
        f'&method={method}&datasetname={data_set}' +
        f'&{params}&ResultFormat=JSON'
    ).json()

In [ ]:
df_data = pd.DataFrame(data["BEAAPI"]['Results']['Data'])
df_data = df_data.set_index('GeoName')
df_data = df_data.sort_values('DataValue', ascending=False)
df_data.head()

In [ ]:
method = "GetData"
data_set = "Regional"
tablename='CAINC4'
geo='COUNTY'
lc='30'
year='2022'

params = f'TableName={tablename}&GeoFips={geo}&LineCode={lc}&Year={year}'

In [ ]:
data_two = requests.get(
        f'https://apps.bea.gov/api/data?&UserID={beakey}' +
        f'&method={method}&datasetname={data_set}' +
        f'&{params}&ResultFormat=JSON'
    ).json()

In [ ]:
df_data_two = pd.DataFrame(data_two["BEAAPI"]['Results']['Data'])
df_data_two.head()

In [ ]:
county_df = df_data_two[['GeoName','DataValue','TimePeriod', 'GeoFips']]
county_df.head()

In [ ]:
method = "GetData"
data_set = "Regional"
tablename='CAINC4'
geo='COUNTY'
lc='30'
year='2022'

years = [year for year in range(2010,2023)]
def call_api(year):
    time.sleep(5)
    print(year)
    params = f'TableName={tablename}&GeoFips={geo}&LineCode={lc}&Year={year}'
    result = requests.get(
            f'https://apps.bea.gov/api/data?&UserID={beakey}' +
            f'&method={method}&datasetname={data_set}' +
            f'&{params}&ResultFormat=JSON'
        ).json()
    df = pd.DataFrame(result["BEAAPI"]['Results']['Data'])
    df = df.sort_values('DataValue', ascending=False)
    df = df[['GeoName','DataValue','TimePeriod', 'GeoFips']]
    # LEAVING GeoName as is so county and state can be changed outside here
    df = df.rename(columns={'DataValue':'Income','TimePeriod':'Year'})
    return df


In [ ]:
results = [call_api(year) for year in years]


In [ ]:
for result in results:
    print(result.head(2))

In [ ]:
county_merged_df = pd.concat(results, ignore_index=True)
county_merged_df['Income'] = pd.to_numeric(county_merged_df['Income'])
county_merged_df.head()


In [ ]:
pivot_merged_df = county_merged_df.pivot(index=['GeoFips','GeoName'], columns='Year', values='Income')
pivot_merged_df.head()

In [ ]:
pct_change_df = pivot_merged_df.pct_change(axis='columns') * 100
pct_change_df = pct_change_df.reset_index()
pct_change_df.to_csv('./files/data/pct_change.csv', index=False)
pct_change_df.head()


In [ ]:
selected_years_10_22 = pivot_merged_df[['2010','2022']]
change_10_22 = selected_years_10_22.pct_change(axis='columns')
change_10_22.reset_index()
change_10_22.to_csv('./files/data/selected_change_10_20.csv')
change_10_22.head()


In [ ]:
method = "GetData"
data_set = "Regional"
tablename='SAINC1'
geo='STATE'
lc='3'

years = [year for year in range(2010,2023)]

results_state = [call_api(year) for year in years]


In [ ]:
state_merged_df = pd.concat(results_state, ignore_index=True)
state_merged_df['Income'] = pd.to_numeric(state_merged_df['Income'])
state_merged_df.head()
pivot_state_merged_df = state_merged_df.pivot(index=['GeoFips','GeoName'], columns='Year', values='Income')
pivot_state_merged_df.head()

In [ ]:
state_selected_years_10_22 = pivot_state_merged_df[['2019','2022']]
state_change_10_22 = state_selected_years_10_22.pct_change(axis='columns') * 100
state_change_10_22 = state_change_10_22.reset_index()
state_change_10_22_drop = state_change_10_22.drop(index=0)
state_change_10_22_drop.to_csv('./files/data/state_selected_change_10_20.csv')
state_change_10_22_drop.head()